In [2]:
import tango
import time
import json
import numpy as np

from ska_control_model import (
    AdminMode,
    CommunicationStatus,
    HealthState,
    PowerState,
    ResultCode,
    TaskStatus,
    SimulationMode,
    TestMode,
)

# for time conversion
from datetime import datetime,timezone
RFC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"
# define devices
controller = tango.DeviceProxy('low-mccs/control/control')
sa1 = tango.DeviceProxy('low-mccs/subarray/01')
sa2 = tango.DeviceProxy('low-mccs/subarray/02')
sab1 = tango.DeviceProxy('low-mccs/subarraybeam/01')
sab2 = tango.DeviceProxy('low-mccs/subarraybeam/02')
st1 = tango.DeviceProxy('low-mccs/station/001')
st2 = tango.DeviceProxy('low-mccs/station/002')
sb1 = tango.DeviceProxy('low-mccs/beam/001')
sb2 = tango.DeviceProxy('low-mccs/beam/002')
sb3 = tango.DeviceProxy('low-mccs/beam/003')
sb4 = tango.DeviceProxy('low-mccs/beam/004')
spsst1 = tango.DeviceProxy('low-mccs/spsstation/ci-1')
spsst2 = tango.DeviceProxy('low-mccs/spsstation/ci-2')
sr1 = tango.DeviceProxy('low-mccs/subrack/ci-1-1')
sr2 = tango.DeviceProxy('low-mccs/subrack/ci-2-1')
t1 = tango.DeviceProxy('low-mccs/tile/ci-1-13')
t2 = tango.DeviceProxy('low-mccs/tile/ci-1-17')
t3 = tango.DeviceProxy('low-mccs/tile/ci-2-13')
t4 = tango.DeviceProxy('low-mccs/tile/ci-2-17')
devices = [spsst1, spsst2, sr1, sr2, t1, t2, t3, t4, st1, st2, 
           sb1, sb2, sb3, sb4, sab1, sab2, sa1, sa2,]
hwdevices = [spsst1, spsst2, sr1, sr2, t1, t2, t3, t4]
sps_stations = [spsst1, spsst2]

<title> Allocate and deallocate resources</title>

1) Assess that the controller has no assigned resources to subarray 1, neither has the subarray

2) Allocate resources to the subarray and check that these have been allocated in the controller, and assigned to the subarray.
    Subarray is composed of two subarray beams, 32 channels and 2 apertures each. 
    Station 1 hosts the 2 apertures of beam 1 and one of beam 2. Station 2 hosts the 2nd aperture of beam 2.

In [5]:
print(f"Controller resources allocated to subarray 1: {controller.GetAssignedResources(1)}")
print(f"Subarray resources\n- Stations:       {sa1.stationFQDNs}")
print(f"- Subarray beams: {sa1.subarraybeamfqdns}")
print(f"- Station beams:  {sa1.stationbeamfqdns}")
print(f"- All assigned resources: {json.loads((sa1.assignedresources))}")

Controller resources allocated to subarray 1: {}
Subarray resources
- Stations:       ()
- Subarray beams: ()
- Station beams:  ()
- All assigned resources: {'interface': 'https://schema.skao.int/ska-low-mccs-assignedresources/1.0', 'subarray_beam_ids': [], 'station_beam_ids': [], 'station_ids': [], 'apertures': [], 'channels': [0]}


In [4]:
args = {
    "subarray_id": 1, 
    "subarray_beams": [
        {
            "subarray_beam_id": 1, 
            "apertures": [
                {
                    "station_id": 1,
                    "aperture_id": "AP001.01"
                },
                {
                    "station_id": 1,
                    "aperture_id": "AP001.02"
                },
            ], 
            "number_of_channels": 32
        },
        {
            "subarray_beam_id": 2, 
            "apertures": [
                {
                    "station_id": 1,
                    "aperture_id": "AP001.03"
                },
                {
                    "station_id": 2,
                    "aperture_id": "AP002.04"
                },
            ], 
            "number_of_channels": 32
        },
    ]}
controller.Allocate(json.dumps(args))

[array([2], dtype=int32), ['1698683688.8465571_160872017842122_Allocate']]

In [5]:
print(f"Controller resources allocated to subarray 1: {controller.GetAssignedResources(1)}")
print(f"Subarray resources\n- Stations:       {sa1.stationFQDNs}")
print(f"- Subarray beams: {sa1.subarraybeamfqdns}")
print(f"- Station beams:  {sa1.stationbeamfqdns}")
print(f"- All assigned resources: {json.loads((sa1.assignedresources))}")

Controller resources allocated to subarray 1: {"subarray_beams": ["low-mccs/subarraybeam/01", "low-mccs/subarraybeam/02"], "station_beams": ["low-mccs/beam/001", "low-mccs/beam/002", "low-mccs/beam/003", "low-mccs/beam/004"]}
Subarray resources
- Stations:       ('low-mccs/station/001', 'low-mccs/station/002')
- Subarray beams: ('low-mccs/subarraybeam/01', 'low-mccs/subarraybeam/02')
- Station beams:  ('low-mccs/beam/001', 'low-mccs/beam/002', 'low-mccs/beam/003', 'low-mccs/beam/004')
- All assigned resources: {'interface': 'https://schema.skao.int/ska-low-mccs-assignedresources/1.0', 'subarray_beam_ids': ['1', '2'], 'station_beam_ids': ['1', '2', '3', '4'], 'station_ids': ['1', '2'], 'apertures': ['AP001.01', 'AP001.02', 'AP001.03', 'AP002.04'], 'channels': [64]}


In [31]:
sa_configuration = {
    "subarray_beams": [
        {
            "subarray_beam_id": 1,
            "update_rate": 5.0,
            "logical_bands": [
                {
                    "start_channel": 66,
                    "number_of_channels": 16
                },
                {
                    "start_channel": 262,
                    "number_of_channels": 16
                }
            ],
            "sky_coordinates": {
                "reference_frame": "ICRS",
                "timestamp": "2023-04-05T12:34:56.000Z",
                "c1": 123.45,
                "c1_rate": 0.0,
                "c2": -12.34,
                "c2_rate": 0.0,
            },
            "apertures": [
                {"aperture_id": "AP001.01"},
                {"aperture_id": "AP001.02"},
            ]
        },
        {
            "subarray_beam_id": 2,
            "update_rate": 10.0,
            "logical_bands": [
                {
                    "start_channel": 166,
                    "number_of_channels": 16
                },
                {
                    "start_channel": 200,
                    "number_of_channels": 16
                }
            ],
            "sky_coordinates": {
                "reference_frame": "ICRS",
                "timestamp": "2023-04-05T12:34:56.000Z",
                "c1": 234.45,
                "c1_rate": 0.001,
                "c2": 23.34,
                "c2_rate": 0.001,
            },
            "apertures": [
                {"aperture_id": "AP001.03"},
                {"aperture_id": "AP002.04"},
            ]
        },
    ]
}
sa1.Configure(json.dumps(sa_configuration))

[array([2], dtype=int32), ['1698735533.6456742_122779193411418_Configure']]

Check that the configuration arrived to the station beams. Check that: 
1) Each Station beam has correct aperture, station and pointing
2) Each Station beam has built a table of beamformer channel blocks. Table entry specifies:
    1) table entry (station channel block)
    2) start physical channel. 0 means unused
    3) hardware beam used by station
    4) subarray ID
    5) start subarray logical channel
    6) subarray beam ID
    7) substation ID
    8) aperture ID
3) Each Station has assembled the beamformer table from all subarray beams

In [32]:
for sb in [sb1, sb2, sb3, sb4]:
    print(f"beam {sb.beamId}: subarray beam {sb.subarrayId}:{sb.logicalBeamId} ID {sb.apertureId} assigned to station {sb.stationId} ({sb.stationFqdn})")
    print(f"Pointing ({sb.pointingReferenceFrame}) {sb.desiredPointing} at time {sb.pointingTimestamp} - HW beam {sb.hardwareBeamId}")
    print(f"Channel table: \n{sb.channels}")
    print(f"Beam {sb.name()} is in state {sb.obsState}")
for st in [st1, st2]:
    print(f"{st.name()}: {st.numberOfChannels} total channels. Channel table:\n{st.beamformerTable}") 

beam 1: subarray beam 1:1 ID AP001.01 assigned to station 1 (low-mccs/station/001)
Pointing (ICRS) [123.45   0.   -12.34   0.  ] at time 2023-04-05T12:34:56.000Z - HW beam 0
Channel table: 
[[  0  66   0   1   0   1   1 101]
 [  1  74   0   1   8   1   1 101]
 [  2 262   0   1  16   1   1 101]
 [  3 270   0   1  24   1   1 101]]
Beam low-mccs/beam/001 is in state 4
beam 2: subarray beam 1:1 ID AP001.02 assigned to station 1 (low-mccs/station/001)
Pointing (ICRS) [123.45   0.   -12.34   0.  ] at time 2023-04-05T12:34:56.000Z - HW beam 1
Channel table: 
[[  4  66   1   1   0   1   2 102]
 [  5  74   1   1   8   1   2 102]
 [  6 262   1   1  16   1   2 102]
 [  7 270   1   1  24   1   2 102]]
Beam low-mccs/beam/002 is in state 4
beam 3: subarray beam 1:2 ID AP001.03 assigned to station 1 (low-mccs/station/001)
Pointing (ICRS) [2.3445e+02 1.0000e-03 2.3340e+01 1.0000e-03] at time 2023-04-05T12:34:56.000Z - HW beam 2
Channel table: 
[[  8 166   2   1  32   2   3 103]
 [  9 174   2   1  40  

1) Deconfigure and reconfigure the subarray
2) Reconfigure an aready configured subarray
3) Deconfigure and deallocate everything
4) Turn off a TPM. The controller goes in Unknown state (partly ON, partly Off)

In [50]:
scan_id = 1234
start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+2), RFC_FORMAT)
sa1.scan(json.dumps({'scan_id': scan_id, 'start_time': start_time}))

[array([2], dtype=int32), ['1698737836.743625_172993090082020_Scan']]

In [ ]:
sa1.endscan()

In [13]:
sa1.end()

[array([2], dtype=int32), ['1698684362.2930324_29537374896676_End']]

In [58]:
controller.Release('{}')

[array([0], dtype=int32), ['Release command completed OK']]

In [3]:
print(spsst1.beamformertable.reshape(-1,7))

[[128   0   1   0   0   0   0]]


In [5]:
print(t1.beamformertable.reshape(-1,7))

[[128   0   1   0   0   0   0]]
